In [ ]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [ ]:
import httpx
import pandas as pd

from urllib.parse import urljoin

from will_it_saturate.servers import (
    FastAPIUvicornServer,
    NginxDockerServer,
    DjangoGunicornWSGIServer,
)
from will_it_saturate.epochs import Epoch
from will_it_saturate.hosts import Host, HostDetails

from will_it_saturate.results import Result
from will_it_saturate.files import BenchmarkFile
from will_it_saturate.control.client import ControlClient
from will_it_saturate.repositories import SqliteRepository, register_default_tables
from will_it_saturate.clients import (
    HttpxClient,
    AioHttpClient,
    WrkClient,
    BaseClient,
)

<IPython.core.display.Javascript object>

# Remote Benchmark

In [ ]:
# dont_test

# create server control client + server (needed for turning files into urls)
server_control_host = Host(name="localhost", port=8001)
server_control_client = ControlClient(host=server_control_host)
server_details = server_control_client.get_host_details()
server = server_control_client.get_or_create_server(FastAPIUvicornServer())
print(server)

# create files and urls
byte, file_size, duration = 8, 10 ** 7, 3
epoch = Epoch(file_size=file_size, duration=duration)
files = server_control_client.get_or_create_files(epoch)
print(files[:10])
epoch.urls = [server.file_to_url(file) for file in files]

# create control client + measure server
client_control_host = Host(name="localhost", port=8001)
client_control_client = ControlClient(host=client_control_host)
client_details = client_control_client.get_host_details()
benchmark_client = HttpxClient(name="httpx")

result = Result(
    server=server,
    client=benchmark_client,
    server_details=server_details,
    client_details=client_details,
    file_size=epoch.file_size,
    complete_size=epoch.complete_size,
)

protocol='http' name='fastAPI/uvicorn' host='localhost' port=8000
[BenchmarkFile(number=0, base_path='10000000_3_125000000', size=10000000, data_root='data', hostname='localhost', port=8000, checksum='b4d1c4304bd32eb46cd8dfa65a45753d', creator_name='filesystem'), BenchmarkFile(number=1, base_path='10000000_3_125000000', size=10000000, data_root='data', hostname='localhost', port=8000, checksum='8b0c6d2a628bccd4af491f4a78d44123', creator_name='filesystem'), BenchmarkFile(number=2, base_path='10000000_3_125000000', size=10000000, data_root='data', hostname='localhost', port=8000, checksum='65ebdcd690b372939379e72bb4a1cbd2', creator_name='filesystem'), BenchmarkFile(number=3, base_path='10000000_3_125000000', size=10000000, data_root='data', hostname='localhost', port=8000, checksum='db69ed48f9243bb76d03ebe092d4dd29', creator_name='filesystem'), BenchmarkFile(number=4, base_path='10000000_3_125000000', size=10000000, data_root='data', hostname='localhost', port=8000, checksum='faea093a44b

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

result.elapsed = client_control_client.measure(benchmark_client, epoch)
print(result.elapsed)

# result.elapsed = benchmark_client.measure(epoch)

3.2735622909999997


<IPython.core.display.Javascript object>

In [ ]:
# dont_test

result.bytes_per_second

114554105.4865481

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

repository = SqliteRepository.build_repository("test.db")
register_default_tables(repository)
results = repository.tables["result"]

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

result_id = results.add_result(result)
print(result_id)

2


<IPython.core.display.Javascript object>

In [ ]:
# dont_test

results.get_results()

[Result(server=FastAPIUvicornServer(protocol='http', name='fastAPI/uvicorn', host='localhost', port=8000), client=HttpxClient(name='httpx'), server_details=HostDetails(machine_id='C02DR0MZQ6LT', cpuinfo={'python_version': '3.9.4.final.0 (64 bit)', 'cpuinfo_version': [7, 0, 0], 'cpuinfo_version_string': '7.0.0', 'arch': 'X86_64', 'bits': 64, 'count': 8, 'arch_string_raw': 'x86_64', 'vendor_id_raw': 'GenuineIntel', 'brand_raw': 'VirtualApple @ 2.50GHz', 'hz_advertised_friendly': '2.5000 GHz', 'hz_actual_friendly': '1.0013 GHz', 'hz_advertised': [2500000000, 0], 'hz_actual': [1001258125, 0], 'l2_cache_size': 65536, 'l2_cache_line_size': 8192, 'l2_cache_associativity': 8, 'model': 44, 'family': 6, 'flags': ['aes', 'apic', 'clflush', 'cmov', 'cx16', 'cx8', 'de', 'dtes64', 'fpu', 'fxsr', 'lahf_lm', 'mmx', 'monitor', 'pat', 'pclmulqdq', 'pni', 'popcnt', 'pse36', 'sep', 'ss', 'sse', 'sse2', 'sse4_1', 'sse4_2', 'ssse3', 'tsc']}), client_details=HostDetails(machine_id='C02DR0MZQ6LT', cpuinfo={'p

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

pd.DataFrame([r.dict_with_properties() for r in results.get_results()])

,server,client,server_details,client_details,file_size,elapsed,complete_size,file_size_h,bytes_per_second,bytes_per_second_h
0,"{'protocol': 'http', 'name': 'fastAPI/uvicorn'...",{'name': 'httpx'},"{'machine_id': 'C02DR0MZQ6LT', 'cpuinfo': {'py...","{'machine_id': 'C02DR0MZQ6LT', 'cpuinfo': {'py...",10000000,3.253347,375000000,9.54MB,1.152659e+08,109.93MB
1,"{'protocol': 'http', 'name': 'fastAPI/uvicorn'...",{'name': 'httpx'},"{'machine_id': 'C02DR0MZQ6LT', 'cpuinfo': {'py...","{'machine_id': 'C02DR0MZQ6LT', 'cpuinfo': {'py...",10000000,3.273562,375000000,9.54MB,1.145541e+08,109.25MB


<IPython.core.display.Javascript object>

In [ ]:
# dont_test

pd.DataFrame([r.dict_for_pandas() for r in results.get_results()])

,server,client,server_host,client_host,elapsed,file_size,file_size_h,complete_size,bytes_per_second,bytes_per_second_h
0,fastAPI/uvicorn,httpx,C02DR0MZQ6LT,C02DR0MZQ6LT,3.253347,10000000,9.54MB,375000000,1.152659e+08,109.93MB
1,fastAPI/uvicorn,httpx,C02DR0MZQ6LT,C02DR0MZQ6LT,3.273562,10000000,9.54MB,375000000,1.145541e+08,109.25MB


<IPython.core.display.Javascript object>

In [ ]:
# dont_test


repo = SqliteRepository.build_repository("foobar.sqlite")

<IPython.core.display.Javascript object>

In [ ]:
def run_epoch(server, client, epoch):
    control_base = "http://localhost:8001/"
    control_epochs = urljoin(control_base, "epochs/")
    r = httpx.post(control_epochs, json=epoch.dict())

    for bfile in r.json()["files"]:
        kwargs = {k: v for k, v in bfile.items() if k != "creator"}
        epoch.files.append(BenchmarkFile(**kwargs))

    control_servers = urljoin(control_base, "servers/")
    r = httpx.post(control_servers, json=server.dict())
    server = server.__class__(**r.json())

    epoch.urls = [server.file_to_url(file) for file in epoch.files]

    result = Result(
        server=server.name,
        client=client.name,
        file_size=epoch.file_size,
        complete_size=epoch.complete_size,
    )

    result.elapsed = client.measure(epoch)
    return result

In [ ]:
# dont_test

byte, file_size, duration = 8, 10 ** 7, 3
epoch = Epoch(file_size=file_size, duration=duration)

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

server = FastAPIUvicornServer()
client = HttpxClient(name="httpx")

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

result = run_epoch(server, client, epoch)

measure


<IPython.core.display.Javascript object>

In [ ]:
# dont_test

result.bytes_per_second

52162649.75030322

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

byte = 8
gigabit = 10 ** 9
bandwidth = gigabit / byte
duration = 3
file_size = 10 ** 7

epoch = Epoch(file_size=file_size, duration=duration)

control_base = "http://localhost:8001/"
control_epochs = urljoin(control_base, "epochs/")

r = httpx.post(control_epochs, json=epoch.dict())

for bfile in r.json()["files"]:
    kwargs = {k: v for k, v in bfile.items() if k != "creator"}
    epoch.files.append(BenchmarkFile(**kwargs))


server = FastAPIUvicornServer()
control_servers = urljoin(control_base, "servers/")
r = httpx.post(control_servers, json=server.dict())
server = FastAPIUvicornServer(**r.json())

epoch.urls = [server.file_to_url(file) for file in epoch.files]

client = HttpxClient(name="httpx")


result = Result(
    server=server.name,
    client=client.name,
    file_size=epoch.file_size,
    complete_size=epoch.complete_size,
)

result.elapsed = client.measure(epoch)
result

measure


Result(server='fastAPI/uvicorn', client='httpx', file_size=10000000, elapsed=6.916868163999999, complete_size=375000000)

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

result.bytes_per_second

54215288.06226934

<IPython.core.display.Javascript object>

# Local Benchmark

In [ ]:
# dont_test

byte = 8
gigabit = 10 ** 9
bandwidth = gigabit / byte
duration = 3
file_size = 10 ** 7


# server = DjangoGunicornWSGIServer()
server = FastAPIUvicornServer()
# client = AioHttpClient(name="aiohttp")
client = HttpxClient(name="httpx")
# repository = SqliteRepository.build_repository("foo.db")

epoch = Epoch(file_size=file_size, duration=duration)

epoch.create_files()

result = Result(
    server=server.name,
    client=client.name,
    file_size=epoch.file_size,
    complete_size=epoch.complete_size,
)

if not server.started:
    server.start

result.elapsed = client.measure(epoch)

ValidationError: 2 validation errors for Result
server
  value is not a valid dict (type=type_error.dict)
client
  value is not a valid dict (type=type_error.dict)

<IPython.core.display.Javascript object>